#### Score: 30.4519

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy import stats

In [3]:
X_train = pd.read_csv('data_dengue/dengue_features_train.csv', index_col=None)
df_y_train = pd.read_csv('data_dengue/dengue_labels_train.csv', index_col=None)
X_test = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)

In [4]:
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
X_train['week_start_date'] = X_train['week_start_date'] - pd.to_datetime('1980-01-01')
X_train['week_start_date'] = pd.to_numeric(X_train['week_start_date'])/100000000000
X_train_new = X_train.fillna(X_train.mean())
X_train_sj=X_train_new[X_train_new.city=='sj']
X_train_iq=X_train_new[X_train_new.city=='iq']
X_train_sj = X_train_sj.drop(['city'], axis=1)
X_train_iq = X_train_iq.drop(['city'], axis=1)

In [5]:
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])
X_test['week_start_date'] = X_test['week_start_date'] - pd.to_datetime('1980-01-01')
X_test['week_start_date'] = pd.to_numeric(X_test['week_start_date'])/100000000000
X_test_new = X_test.fillna(X_test.mean())
X_test_sj=X_test_new[X_test_new.city=='sj']
X_test_iq=X_test_new[X_test_new.city=='iq']
X_test_sj = X_test_sj.drop(['city'], axis=1)
X_test_iq = X_test_iq.drop(['city'], axis=1)

In [6]:
X_train_sj_float = X_train_sj.astype('float64')
X_train_iq_float = X_train_iq.astype('float64')
X_test_sj_float = X_test_sj.astype('float64')
X_test_iq_float = X_test_iq.astype('float64')

In [7]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_sj_scaled = min_max_scaler.fit_transform(X_train_sj_float)
X_train_iq_scaled = min_max_scaler.fit_transform(X_train_iq_float)
X_test_sj_scaled = min_max_scaler.transform(X_test_sj_float)
X_test_iq_scaled = min_max_scaler.transform(X_test_iq_float)

In [8]:
train_1s_sj = X_train_sj_scaled.reshape(X_train_sj_scaled.shape[0],1,X_train_sj_scaled.shape[1])
train_1s_iq = X_train_iq_scaled.reshape(X_train_iq_scaled.shape[0],1,X_train_iq_scaled.shape[1])

In [9]:
train_1s_sj.shape

(936, 1, 23)

In [10]:
data_y_1s_sj = np.array(df_y_train[df_y_train.city=='sj']["total_cases"])
data_y_1s_iq = np.array(df_y_train[df_y_train.city=='iq']["total_cases"])
data_y_1s_sj.shape

(936,)

In [13]:
model_sj = Sequential()
model_sj.add(Bidirectional(GRU(50, activation='relu', return_sequences=True, input_shape=(1, 23))))
model_sj.add(Dropout(0.2))
model_sj.add(Bidirectional(GRU(100, activation='relu', return_sequences=False)))
model_sj.add(Dropout(0.2))
model_sj.add(Dense(1, activation = "linear"))
model_sj.compile(optimizer='adam', loss='mae')

In [14]:
%%time
model_sj.fit(train_1s_sj, data_y_1s_sj, epochs=400, verbose=1)


Epoch 1/400
936/936 [==============================] - 5s 5ms/step - loss: 33.3241
Epoch 2/400
936/936 [==============================] - 0s 436us/step - loss: 26.9051
Epoch 3/400
936/936 [==============================] - 0s 461us/step - loss: 24.4331
Epoch 4/400
936/936 [==============================] - 0s 456us/step - loss: 24.2175
Epoch 5/400
936/936 [==============================] - 0s 476us/step - loss: 23.9007
Epoch 6/400
936/936 [==============================] - 0s 444us/step - loss: 23.6902
Epoch 7/400
936/936 [==============================] - 0s 456us/step - loss: 23.5861
Epoch 8/400
936/936 [==============================] - 0s 455us/step - loss: 23.3891
Epoch 9/400
936/936 [==============================] - 0s 454us/step - loss: 23.1834
Epoch 10/400
936/936 [==============================] - 0s 450us/step - loss: 23.2937
Epoch 11/400
936/936 [==============================] - 0s 454us/step - loss: 23.0914
Epoch 12/400
936/936 [==============================] - 1s 579us

In [15]:
model_iq = Sequential()
model_iq.add(Bidirectional(GRU(50, activation='relu', return_sequences=True, input_shape=(1, 23))))
model_iq.add(Dropout(0.2))
model_iq.add(Bidirectional(GRU(100, activation='relu', return_sequences=False)))
model_iq.add(Dropout(0.2))
model_iq.add(Dense(1, activation = "linear"))
model_iq.compile(optimizer='adam', loss='mae')

In [16]:
%%time
model_iq.fit(train_1s_iq, data_y_1s_iq, epochs=400, verbose=0)

Wall time: 1min 40s


In [17]:
data_1s_test_sj = X_test_sj_scaled.reshape(X_test_sj_scaled.shape[0],1,X_test_sj_scaled.shape[1])
data_1s_test_iq = X_test_iq_scaled.reshape(X_test_iq_scaled.shape[0],1,X_test_iq_scaled.shape[1])
data_1s_test_sj.shape

(260, 1, 23)

In [18]:
%%time
predict_sj = model_sj.predict(data_1s_test_sj)

Wall time: 1.34 s


In [19]:
%%time
predict_iq = model_iq.predict(data_1s_test_iq)

Wall time: 1.13 s


In [20]:
X_test_predict = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)
result_columns_sj = X_test[X_test.city=='sj'][['city','year','weekofyear']]
result_cases_sj = pd.DataFrame(predict_sj)
result_cases_sj.columns = ['total_cases']
result_df_sj = pd.concat([result_columns_sj, result_cases_sj], axis=1)
result_df_sj['total_cases'] = result_df_sj['total_cases'].round().astype(int)
result_df_sj.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,9
1,sj,2008,19,11
2,sj,2008,20,10
3,sj,2008,21,9
4,sj,2008,22,10


In [21]:
result_columns_iq = X_test[X_test.city=='iq'][['city','year','weekofyear']]
result_cases_iq = pd.DataFrame(predict_iq)
result_cases_iq.columns = ['total_cases']
result_columns_iq = result_columns_iq.reset_index()
result_df_iq = pd.concat([result_columns_iq, result_cases_iq], axis=1)
result_df_iq['total_cases'] = result_df_iq['total_cases'].round().astype(int)
result_df_iq = result_df_iq.drop(['index'], axis=1)
result_df_iq.head()

,city,year,weekofyear,total_cases
0,iq,2010,26,1
1,iq,2010,27,1
2,iq,2010,28,2
3,iq,2010,29,3
4,iq,2010,30,2


In [22]:
result = pd.concat([result_df_sj,result_df_iq], axis=0)
result.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,9
1,sj,2008,19,11
2,sj,2008,20,10
3,sj,2008,21,9
4,sj,2008,22,10


In [23]:
result.to_csv('gru_2_city_dropout.csv', index=False)